In [1]:
"""
Stochastic Calculus

Week 4. Risk neutral pricing
"""

import pandas as pd
import numpy as np
import seaborn as sns
from numpy import exp, log, sqrt, max, min, mean

def get_time(T=1, n=1000):
    """    
    Parameters
    ----------
    T : positive float, optional
        Final moment of time. The default is 1.
    n : integer, optional
        Number of increments. The default is 1000.

    Returns
    -------
    Vector of time points.
    """
    return np.linspace(start=0, stop=T, num=n+1)
    
def get_wiener_increments(T=1, n=1000, seed=None):
    """    
    Parameters
    ----------
    T : positive float, optional
        Final moment of time. The default is 1.
    n : integer, optional
        Number of increments. The default is 1000.
    seed : Initial seed for random number generation.
        The default is None.

    Returns
    -------
    Vector of Wiener process increments.
    """
    sd = sqrt(T/n)
    rng = np.random.default_rng(seed)
    delta_W = rng.normal(loc=0, scale=sd, size=n)
    return delta_W


def get_wiener_trajectory(T=1, n=1000, seed=None):
    """    
    Parameters
    ----------
    T : positive float, optional
        Final moment of time. The default is 1.
    n : integer, optional
        Number of increments. The default is 1000.
    seed : Initial seed for random number generation.
        The default is None.

    Returns
    -------
    Vector of (n+1) Wiener process values
    """
    delta_w = get_wiener_increments(T=T, seed=seed, n=n)
    w = np.zeros(n+1)
    w[1:(n+1)] = np.cumsum(delta_w)
    return w



def get_integral_trajectory(integrand_fun, T=1, n=1000, seed=None):
    """    
    Parameters
    ----------
    integrand_fun : function of w and t to integrate,
        lambda w, t : w ** 3 as example.
    T : positive float, optional
        Final moment of time. The default is 1.
    n : integer, optional
        Number of increments. The default is 1000.
    seed : Initial seed for random number generation.
        The default is None.

    Returns
    -------
    Integral trajectory.
    """

    delta_w = get_wiener_increments(T=T, seed=seed, n=n)
    w = np.zeros(n+1)
    w[1:(n+1)] = np.cumsum(delta_w)
    t = get_time(T=T, n=n)
    integrand = integrand_fun(w, t)
    
    delta_integral = integrand[0:n] * delta_w
    
    integral = np.zeros(n+1)
    integral[0] = 0
        
    integral[1:(n+1)] = np.cumsum(delta_integral)

    return integral


def get_sde_trajectory(integrand_dw, integrand_dt, T=1, n=1000, seed=None, y0=0):
    """    
    Parameters
    ----------
    integrand_dw : function of y, w and t to integrate,
        lambda y, w, t : 0.02 as example.
    integrand_dw : function of y, w and t to integrate,
        lambda y, w, t : (0.05 - y) as example.
    T : positive float, optional
        Final moment of time. The default is 1.
    n : integer, optional
        Number of increments. The default is 1000.
    seed : Initial seed for random number generation.
        The default is None.

    Returns
    -------
    SDE solution trajectory.
    """
        
    delta_w = get_wiener_increments(T=T, seed=seed, n=n)
    w = np.zeros(n+1)
    w[1:(n+1)] = np.cumsum(delta_w)
    
    t = get_time(T=T, n=n)
    dt = T/n
    
    y = np.zeros(n+1)
    y[0] = y0
    
    for i in range(n):
        delta_y = integrand_dw(y[i], w[i], t[i]) * delta_w[i] + integrand_dt(y[i], w[i], t[i]) * dt
        y[i+1] = y[i] + delta_y

    return y



In [9]:
# The contract pays you S_T if S_u was never above 120 for u in [0;T]
# and 0 otherwise.
# Estimate the price of the contract.
S0 = 100
r = 0.05
sigma = 0.2
T = 3

In [10]:
n = 1000
n_sim = 10000

final_price = np.zeros(n_sim)

In [11]:
for i in range(n_sim):
  w = get_wiener_trajectory(T=T, n=n)
  t = get_time(T=T, n=n)
  S = S0 * exp((r - sigma ** 2 / 2) * t + sigma * w)
  final_price[i] = S[n] * (max(S) < 120)

In [12]:
exp(- r * T) * mean(final_price)

23.94062976381009